In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import mlflow


from scipy.stats import (
                         shapiro,
                         levene,
                         mannwhitneyu,
                         ttest_ind,
                         chi2_contingency,
                         fisher_exact)

from sklearn.preprocessing import (
                            MinMaxScaler,
                            OneHotEncoder,
                            LabelEncoder
                            )

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from statsmodels.discrete.discrete_model import Logit
from statsmodels.stats.outliers_influence import variance_inflation_factor

pd.set_option('display.max_columns', None)

In [2]:
root = Path('Attrition_Application/artifacts/data_ingestion_artifact/Attrition.csv')
dataset_path = Path.joinpath(Path('/').resolve(),root)

In [3]:
df = pd.read_csv(dataset_path)

In [13]:
significant_features = ['JobInvolvement',
 'JobLevel',
 'StockOptionLevel',
 'JobRole',
 'MaritalStatus',
 'OverTime',
 'DistanceFromHome',
 'HourlyRate',
 'MonthlyRate',
 'NumCompaniesWorked',
 'PercentSalaryHike',
 'TrainingTimesLastYear',
 'YearsSinceLastPromotion',
 'Attrition']

In [35]:
sacle_features= [
                 'HourlyRate',
                 'MonthlyRate',
                 'PercentSalaryHike',
                 'TrainingTimesLastYear',
                 'YearsSinceLastPromotion',
                 'JobInvolvement',
                 'JobLevel',
                 'StockOptionLevel',
                 'DistanceFromHome',
                 'NumCompaniesWorked',
                 ]
ohe_features = ['OverTime',
                'MaritalStatus',
                'JobRole'
                ]
label_encode_features = ['Attrition']

In [15]:
df = df[significant_features]

In [ ]:
 df.head()

,JobInvolvement,JobLevel,StockOptionLevel,JobRole,MaritalStatus,OverTime,DistanceFromHome,HourlyRate,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,TrainingTimesLastYear,YearsSinceLastPromotion,Attrition
0,3,2,0,Sales Executive,Single,Yes,1,94,19479,8,11,0,0,Yes
1,2,2,1,Research Scientist,Married,No,8,61,24907,1,23,3,1,No
2,2,1,0,Laboratory Technician,Single,Yes,2,92,2396,6,15,3,0,Yes
3,3,1,0,Research Scientist,Married,Yes,3,56,23159,1,11,3,3,No
4,3,1,1,Laboratory Technician,Married,No,2,40,16632,9,12,3,2,No


In [82]:
from imblearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression

In [46]:
X_train,X_test,y_train,y_test = train_test_split(df.drop('Attrition',axis=1),df['Attrition'],test_size=0.2)

In [43]:
ColumnTransformerStep = ColumnTransformer([
    ('MinMaxScaler',MinMaxScaler(),sacle_features),
    ('OneHotEncoder',OneHotEncoder(),ohe_features)
])

In [49]:
y_train_transformed = LabelEncoder().fit_transform(y_train)

In [89]:
pipe = Pipeline([('cl',ColumnTransformerStep),('smt',SMOTE())])

In [92]:
pipe.get_feature_names_out

<bound method Pipeline.get_feature_names_out of Pipeline(steps=[('cl',
                 ColumnTransformer(transformers=[('MinMaxScaler',
                                                  MinMaxScaler(),
                                                  ['HourlyRate', 'MonthlyRate',
                                                   'PercentSalaryHike',
                                                   'TrainingTimesLastYear',
                                                   'YearsSinceLastPromotion',
                                                   'JobInvolvement', 'JobLevel',
                                                   'StockOptionLevel',
                                                   'DistanceFromHome',
                                                   'NumCompaniesWorked']),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(),
                                                  ['OverTim

In [ ]:
X_train_res,y_train_res = pipe.fit_resample(train,y_train_transformed)

In [96]:
y_train_res.reshape(-1,1)

array([[0],
       [0],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [93]:
X_train_res.shape

(1960, 24)